# Application

In this final notebook, we apply the previously gathered knowledge to extract the pin configuration for the blocks 8A, 8B, 8C and 8D in the bitstream retreived from the on-board Flash. We expect to find a 72-bit DDR3 interface.

In [1]:
import csv
import enum
import os
from collections import namedtuple

import numpy as np

from JicBitstream import JicBitstream, JicBitstreamZip
from IOclassifier import IOclassifier, IOSTD_REL_TO_PU, IOclassifierOut

%matplotlib notebook

In [2]:
pin_list = open("../../resources/pin_list_5SGSMD5K1F40C1_8AD.txt", "r").readlines()
pin_list = [pin.strip() for pin in pin_list]

In [3]:
jic_val1 = JicBitstream("../../bitstreams/decomp/factory_decompress1.jic")
iocls = IOclassifier()


@enum.unique
class TermColor(enum.Enum):
    RED = "\x1b[31m"
    GREEN = "\x1b[32m"
    BLUE = "\x1b[34m"
    MAGENTA = "\x1b[35m"
    CYAN = "\x1b[36m"
    BOLD = "\x1b[1m"
    RESET = "\x1b[0m"


def get_color(pin: IOclassifierOut):

    if pin.inp and pin.out and pin.diff:
        return TermColor.RED.value
    elif pin.inp and pin.out:
        return TermColor.BLUE.value
    elif pin.out:
        return TermColor.GREEN.value
    elif pin.inp:
        return TermColor.MAGENTA.value
    elif not pin.inp and not pin.out:
        return TermColor.CYAN.value
    return TermColor.RESET.value


class PinSummary:
    def __init__(self):
        self.dq = []
        self.dqs = []
        self.dm = []
        self.addr = []
        self.unknown = []

    def add_pin(self, pin_name: str, pin_class: IOclassifierOut) -> str:
        if (
            pin_class.inp
            and pin_class.out
            and pin_class.diff
            and pin_class.term == "SSTL, term"
        ):
            cls = "DQS"
            self.dqs.append(pin_name)
        elif (
            pin_class.inp
            and pin_class.out
            and not pin_class.diff
            and pin_class.term == "SSTL, term"
        ):
            cls = "DQ"
            self.dq.append(pin_name)
        elif (
            not pin_class.inp
            and pin_class.out
            and not pin_class.diff
            and pin_class.term == "SSTL, term"
        ):
            cls = "DM"
            self.dm.append(pin_name)
        elif (
            not pin_class.inp
            and pin_class.out
            and not pin_class.diff
            and pin_class.term == "no term"
        ):
            cls = "addr"
            self.addr.append(pin_name)
        elif not pin_class.inp and not pin_class.out:
            cls = "n/c"
        else:
            cls = "?"
            self.unknown.append(pin_name)

        return cls

    def print_info(self):
        print("Pin summary:")
        print(
            f"{TermColor.BOLD.value}nr pins: "
            f"DQ -> {len(self.dq)}, "
            f"DQS -> {len(self.dqs)}, "
            f"DM -> {len(self.dm)}, "
            f"addr -> {len(self.addr)}, "
            f"unknown -> {len(self.unknown)}"
            f"{TermColor.RESET.value}"
        )
        print(f"DQ = {self.dq}")
        print(f"DQS = {self.dqs}")
        print(f"DM = {self.dm}")
        print(f"addr/control = {self.addr}")
        print(f"unknown = {self.unknown}")


y_ = iocls.classify(jic_val1, pin_list)
pin_summary = PinSummary()

for pin, pin_class in zip(pin_list, y_):
    cls = pin_summary.add_pin(pin, pin_class)
    print(
        get_color(pin_class),
        pin,
        pin_class,
        TermColor.BOLD.value,
        cls,
        TermColor.RESET.value,
    )

 K21 IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)  addr 
 J21 IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)  addr 
 M20 IOclassifierOut(inp=False, out=False, pu=True, io_std='2V5', term=None, diff=False)  n/c 
 L20 IOclassifierOut(inp=False, out=True, pu=False, io_std='S', term='SSTL, term', diff=False)  DM 
 K22 IOclassifierOut(inp=False, out=False, pu=True, io_std='2V5', term=None, diff=False)  n/c 
 J22 IOclassifierOut(inp=False, out=False, pu=True, io_std='2V5', term=None, diff=False)  n/c 
 M21 IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)  addr 
 L21 IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)  addr 
 P22 IOclassifierOut(inp=False, out=False, pu=True, io_std='2V5', term=None, diff=False)  n/c 
 N22 IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)  addr 
 N20 

In [4]:
pin_summary.print_info()

Pin summary:
nr pins: DQ -> 72, DQS -> 17, DM -> 10, addr -> 25, unknown -> 5
DQ = ['F20', 'E20', 'G20', 'C20', 'C21', 'E21', 'B20', 'A20', 'H23', 'G23', 'H22', 'C22', 'B22', 'A22', 'B23', 'A23', 'G24', 'F24', 'G25', 'D24', 'C24', 'C25', 'B25', 'A25', 'G26', 'F26', 'H26', 'D27', 'B26', 'A26', 'C26', 'C27', 'E28', 'H28', 'G28', 'C28', 'B28', 'A28', 'B29', 'A29', 'B31', 'A31', 'D30', 'C30', 'F30', 'E31', 'H31', 'G31', 'R32', 'P32', 'T31', 'M33', 'L33', 'L34', 'P34', 'N34', 'D33', 'C33', 'B32', 'A32', 'A34', 'A35', 'A36', 'A37', 'F32', 'G33', 'G32', 'K33', 'J33', 'G34', 'K34', 'J34']
DQS = ['G21', 'F21', 'F23', 'E24', 'E25', 'G27', 'F27', 'H29', 'G29', 'D31', 'C31', 'N32', 'M32', 'E34', 'D34', 'F33', 'E33']
DM = ['L20', 'D21', 'D22', 'D25', 'E27', 'D28', 'E30', 'N33', 'C34', 'H34']
addr/control = ['K21', 'J21', 'M21', 'L21', 'N22', 'N21', 'L24', 'K24', 'P23', 'N23', 'P25', 'N25', 'P26', 'N26', 'M26', 'L26', 'N27', 'K27', 'J27', 'M27', 'L27', 'J29', 'J28', 'L28', 'K28']
unknown = ['M23', '

And here we have it - we have managed to find and classify (almost) all pins which make the DDR3 interface. There is some small clean-up still needed (one of the DQS pins is missing, one DM pin too much, a couple of pins with "unknown" function), but from here one can start to experiment with the real hardware.